# Running the Supervised Optimizer

From start to finish, on pretrained weights


In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import torch
import numpy as np
import os
import json
from Ising import Ising
from model import TransformerModel
from optimizer_supervised import Optimizer

In [26]:
def gpu_setup():
    # Setup for PyTorch:
    if torch.cuda.is_available():
        torch_device = torch.device("cuda")
        print("PyTorch is using GPU {}".format(torch.cuda.current_device()))
    else:
        torch_device = torch.device("cpu")
        print("GPU unavailable; using CPU")

In [27]:
!nvidia-smi

Thu Aug  1 09:17:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:01:00.0  On |                  N/A |
| 64%   52C    P2             124W / 350W |   1693MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
gpu_setup()

PyTorch is using GPU 0


In [1]:
torch.set_default_device("cuda")
torch.set_default_dtype(torch.float32)

NameError: name 'torch' is not defined

In [30]:
import plotly.graph_objects as go
import numpy as np


def plot_tensor(tens, labels, opacity=0.7, size=5):

    x = np.arange(tens.shape[0])
    y = np.arange(tens.shape[1])
    z = np.arange(tens.shape[2])

    xlen = len(x)
    ylen = len(y)
    zlen = len(z)

    print(f"(x, y, z) = ({xlen}, {ylen}, {zlen})")

    X, Y, Z = np.meshgrid(x, y, z)

    color_function = np.vectorize(lambda x, y, z: tens[x, y, z])

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                mode="markers",
                marker=dict(
                    size=size,
                    # color=tens.swapaxes(1, 2)
                    # .swapaxes(0, 2)
                    # .swapaxes(1, 2)
                    # .flatten(),  # set color to an array/list of desired values
                    color=color_function(X, Y, Z).flatten(),
                    colorscale="bupu",  # choose a colorscale
                    opacity=opacity,
                ),
            )
        ]
    )

    fig.update_layout(
        scene=dict(xaxis_title=labels[0], yaxis_title=labels[1], zaxis_title=labels[2]),
    )
    fig.show()

## New Probabilistic Batched Method


In [31]:
system_sizes = torch.arange(15, 15 + 2, 2).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True, get_basis=True) for size in system_sizes]
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 10000
param_range = None
point_of_interest = None
use_SR = False

print("Sizes:\n", system_sizes)
print("Dimensions of parameter space:", param_dim)
print("Number of units in a feedforward layer:", n_hid)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


Sizes:
 tensor([[15]], device='cuda:0')
Dimensions of parameter space: 1
Number of units in a feedforward layer: 32


In [32]:
import math

gaussian_coeff = 1 / math.sqrt(2 * math.pi)
gaussian_mean = 1.0
gaussian_std = 0.05
probability_distribution = lambda param: gaussian_coeff * torch.exp(
    -0.5 * (((param - gaussian_mean) ** 2) / gaussian_std**2)
)

In [33]:
data_dir_path = os.path.join("TFIM_ground_states", "2024-07-24T19-26-39.836")
# data_dir_path = os.path.join("TFIM_ground_states", "2024-07-24T16-19-00.994")

for ham in Hamiltonians:
    ham.load_dataset(
        data_dir_path,
        batch_size=30000,
        samples_in_epoch=100,
        sampling_type="sequential",
    )
    # ham.training_dataset.set_sampling_distribution(probability_distribution)

print("Hamiltonians:", Hamiltonians)

Loaded dataset for system size 15 from TFIM_ground_states/2024-07-24T19-26-39.836/15.arrow.
(h_min, h_step, h_max) = (0.5, 0.01, 1.5).
Hamiltonians: [<Ising.Ising object at 0x74d301df1190>]


In [34]:
Hamiltonians[0].dataset

,N,h,energy,state
0,15,0.50,-15.953170,"[0.6230531142436205, 0.07918341825738834, 0.07..."
1,15,0.51,-15.992388,"[0.6195952254116485, 0.0803763042893551, 0.080..."
2,15,0.52,-16.032444,"[0.6160625240927698, 0.08154485611438121, 0.08..."
3,15,0.53,-16.073342,"[0.6152313284867159, 0.08306336670116583, 0.08..."
4,15,0.54,-16.115088,"[0.6087701876950788, 0.08380647181873989, 0.08..."
...,...,...,...,...
96,15,1.46,-24.555332,"[0.08129764151807829, 0.03547150446859837, 0.0..."
97,15,1.47,-24.685879,"[0.07967117860998968, 0.034997069483799985, 0...."
98,15,1.48,-24.816726,"[0.07810205373098152, 0.03453632521710248, 0.0..."
99,15,1.49,-24.947865,"[0.07658757690991246, 0.03408871756724923, 0.0..."


In [35]:
testmodel = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

results_dir = "results"
paper_checkpoint_name = "ckpt_100000_Ising_32_8_8_0.ckpt"
paper_checkpoint_path = os.path.join(results_dir, paper_checkpoint_name)
checkpoint = torch.load(paper_checkpoint_path)
testmodel.load_state_dict(checkpoint)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [36]:
testmodel.cuda()

TransformerModel(
  (pos_encoder): TQSPositionalEncoding1D(
    (dropout): Dropout(p=0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=32, out_features=32, bias=True)
          (linear_Q): Linear(in_features=32, out_features=32, bias=True)
          (linear_K): Linear(in_features=32, out_features=32, bias=True)
          (linear_V): Linear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=32, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=32, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
    )


In [37]:
from optimizer_supervised_batches import Optimizer

In [38]:
opt = Optimizer(testmodel, Hamiltonians, point_of_interest=point_of_interest)

In [39]:
drmg40path = os.path.join("results", "E_dmrg_40.npy")
dmrg40 = np.load(drmg40path)
dmrg40

array([-39.        , -39.00420011, -39.01680176, -39.03780892,
       -39.06722821, -39.10506893, -39.1513431 , -39.20606547,
       -39.26925361, -39.3409279 , -39.42111169, -39.5098313 ,
       -39.60711613, -39.7129988 , -39.82751523, -39.95070478,
       -40.0826104 , -40.2232788 , -40.37276064, -40.53111075,
       -40.69838833, -40.87465727, -41.05998641, -41.25444989,
       -41.45812755, -41.67110535, -41.89347585, -42.12533881,
       -42.36680181, -42.61798101, -42.879002  , -43.15000084,
       -43.43112524, -43.72253594, -44.02440843, -44.33693501,
       -44.66032733, -44.99481972, -45.34067359, -45.69818378,
       -46.06768807, -46.44958242, -46.84434575, -47.25257988,
       -47.67506782, -48.11283922, -48.56718438, -49.0394836 ,
       -49.53074211, -50.04104421, -50.56943379, -51.11432399,
       -51.67399663, -52.24687977, -52.83162458, -53.4270924 ,
       -54.03231875, -54.64647861, -55.2688584 , -55.8988346 ,
       -56.53585745, -57.1794384 , -57.82914044, -58.48

In [40]:
ising40 = Ising(
    torch.tensor([40]),
    periodic=False,
    get_basis=False,
)

In [41]:
ising40.DMRG(param=0.02)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/tenpy/models/model.py:1946: UserWarning: Invalid type for key "L". Expected <class 'int'>. Got Tensor.
  L = model_params.get('L', 2, int)


(-39.00420011001162,
 <tenpy.models.spins.SpinModel at 0x74d2b0ab5f70>)

In [42]:
dmrg40_h_values = np.linspace(0, 2, 101)
dmrg40_h_values

array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
       0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
       0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
       0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
       0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  , 1.02, 1.04, 1.06, 1.08,
       1.1 , 1.12, 1.14, 1.16, 1.18, 1.2 , 1.22, 1.24, 1.26, 1.28, 1.3 ,
       1.32, 1.34, 1.36, 1.38, 1.4 , 1.42, 1.44, 1.46, 1.48, 1.5 , 1.52,
       1.54, 1.56, 1.58, 1.6 , 1.62, 1.64, 1.66, 1.68, 1.7 , 1.72, 1.74,
       1.76, 1.78, 1.8 , 1.82, 1.84, 1.86, 1.88, 1.9 , 1.92, 1.94, 1.96,
       1.98, 2.  ])

In [43]:
print(len(dmrg40_h_values))
print(len(dmrg40))

101
101


In [44]:
ising40.system_size

tensor([40], device='cuda:0')

In [45]:
oneidx = np.where(dmrg40_h_values == 1.0)[0][0]
oneidx

50

In [46]:
dmrg40[oneidx]

-50.56943379479509

In [48]:
opt.train(
    epochs=1,
    start_iter=0,
    monitor_params=torch.tensor([[1.0]]),
    monitor_hamiltonians=[ising40],
    monitor_energies=torch.tensor([[dmrg40_h_values[oneidx]]])
)

Epoch 0 iter 0 - Loss for system size tensor([15], device='cuda:0') and h-range 0.5-0.5: 0.11009185016155243
compute_observable time: 0.0005548000335693359
compute_observable time: 3.0858829021453857
Sample time: 0.4039294719696045, Eloc time: 3.086578845977783
	param=tensor([1.], device='cuda:0'), system_size=tensor([40], device='cuda:0') - Relative Error: tensor([[51.5510]], device='cuda:0', dtype=torch.float64)
		Energy: (-50.55104446411133+0.0024252498988062143j), Variance: 8.536142559023574e-06, Real: -1.2637760639190674, Imag: 6.0631246014963835e-05
Time breakdown: psi: 0.0041866302490234375, loss: 0.00017142295837402344, backprop: 0.7667841911315918, energy: 3.491574764251709
Epoch 0 iter 1 - Loss for system size tensor([15], device='cuda:0') and h-range 0.5-0.51: 0.11004701256752014
compute_observable time: 0.0005495548248291016
compute_observable time: 2.995652437210083
Sample time: 0.39482808113098145, Eloc time: 2.99642014503479
	param=tensor([1.], device='cuda:0'), system_s

KeyboardInterrupt: 

In [ ]:
Hamiltonians[0].training_dataset.sampled

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [20]:
plot_tensor(
    Hamiltonians[0].training_dataset.sampled.unsqueeze(1).cpu().numpy(),
    ["batch", "system size", "parameter"],
    opacity=0.5,
    size=3,
)

(x, y, z) = (101, 1, 16)


# Old Non-Batched Method


In [ ]:
system_sizes = torch.arange(2, 16 + 1, 2).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True) for size in system_sizes]
param_dim = Hamiltonians[0].param_dim
embedding_size = 32
n_head = 8
n_hid = embedding_size
n_layers = 8
dropout = 0
minibatch = 1000
param_range = None
point_of_interest = None
use_SR = False

Hamiltonians = [Ising(L) for L in system_sizes]
data_dir_path = os.path.join("TFIM_ground_states", "2024-07-24T19-26-39.836")
for ham in Hamiltonians:
    ham.load_dataset(data_dir_path)

print("Sizes:\n", system_sizes)
print("Hamiltonians:", Hamiltonians)
print("Dimensions of parameter space:", param_dim)
print("Number of units in a feedforward layer:", n_hid)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
testmodel = TransformerModel(
    system_sizes,
    param_dim,
    embedding_size,
    n_head,
    n_hid,
    n_layers,
    dropout=dropout,
    minibatch=minibatch,
)

results_dir = "results"
paper_checkpoint_name = "ckpt_100000_Ising_32_8_8_0.ckpt"
paper_checkpoint_path = os.path.join(results_dir, paper_checkpoint_name)
checkpoint = torch.load(paper_checkpoint_path)
testmodel.load_state_dict(checkpoint)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<All keys matched successfully>

In [ ]:
testmodel.cuda()

TransformerModel(
  (pos_encoder): TQSPositionalEncoding1D(
    (dropout): Dropout(p=0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=32, out_features=32, bias=True)
          (linear_Q): Linear(in_features=32, out_features=32, bias=True)
          (linear_K): Linear(in_features=32, out_features=32, bias=True)
          (linear_V): Linear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=32, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=32, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, inplace=False)
      )
    )


In [ ]:
opt = Optimizer(testmodel, Hamiltonians, point_of_interest=point_of_interest)

/home/spandan/anaconda3/envs/tqs2/lib/python3.12/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)
/home/spandan/Projects/tqs/model.py:228: SyntaxWarning: invalid escape sequence '\p'
  """


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
import cProfile
import pstats

In [ ]:
epochs = 3
param_range = torch.tensor([[0.5, 1.5]])
param_step = torch.tensor([0.01])

In [ ]:
testmodel.minibatch = 1000

In [ ]:
# with cProfile.Profile() as pr:
# with torch.autograd.profiler.profile(use_cuda=True) as prof:
opt.train(epochs=epochs, param_range=param_range, param_step=param_step, start_iter=0)

Ran forward for tensor([2], device='cuda:0') spins at point (0.5,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5099999997764826,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5199999995529652,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5299999993294477,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5399999991059303,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5499999988824129,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5599999986588955,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5699999984353781,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5799999982118607,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5899999979883432,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.5999999977648258,)
Ran forward for tensor([2], device='cuda:0') spins at point (0.6099999975413084,)
Ran forward for tensor([2], dev

/home/spandan/Projects/tqs/model.py:228: SyntaxWarning: invalid escape sequence '\p'
  """


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 